# ウェブデータ処理２：WebAPIの活用

画像処理で紹介したような深層学習を使ったアルゴリズムでは、メモリやGPGPUなど高スペックなコンピュータを必要としますが、極めて精度の高い結果を導くことができましたね。   
他にも、マシンのリソースや環境を考えなくてよければ、大規模なデータで高速に計算し、高い精度の結果を導くことのできるアルゴリズムは数多くあります。

アプリケーションを開発する場合は、当然このような高性能のアルゴリズムを使いたいと思いますよね。   
ですが、このようなアルゴリズムは、皆さんのスマートフォンやノートPCでは、スペックが足りなくて動かないかもしれません（例えば深層学習の類はそうなることが多いです）。   
それではどうしたらいいでしょうか？

もしそのアプリの利用を想定しているスマホやノートPCがインターネットにつながっていると想定できるのであれば、
データを高スペックなサーバにインターネット経由で送りつけ、サーバで高度な処理を行った後、
その結果をまたインターネット経由で送り返してもらえばいいです。   
このような機能を実現する仕組みをWeb API (Web Application Programming Interface）と呼びます。   
（もちろん、低スペックマシンでオフラインで動作するようなコンパクトで高性能なアルゴリズムを開発することも重要な課題です！）

Google、Microsoft、Amazon、facebook、IBM、Twitter、Yahoo、docomoなど、Webサービスを提供する大手企業のほとんどはWeb APIを提供しています。   
以下に一例をあげます（リンク先にはWebAPI以外のサービスについても書かれています）。

- [Google Cloud API](https://cloud.google.com/apis/docs/overview?hl=ja)
- [Microsoft Azure API Management](https://azure.microsoft.com/ja-jp/services/api-management/)
- [AWS Amazon API Gateway (WebAPIを構築するサービス）](https://aws.amazon.com/jp/api-gateway/)
- [facebook for developpers](https://developers.facebook.com/)
- [IBM Watson API](https://www.ibm.com/watson/jp-ja/)
- [Twitter](https://developer.twitter.com/)
- [Yahoo! Japan](https://developer.yahoo.co.jp/)
- [docomo](https://dev.smt.docomo.ne.jp/?p=docs.api.index)

第2回で`pandas-datareader`でオンラインで株価を取得しましたが、あれもWeb APIの一種です。   

`pandas-datareader`では無料でサービスを利用できましたが、Web APIは基本的に有料のサービスです。   
各社のAPIを利用する場合は、まずそのサイトでアカウントを作成し、場合によってはクレジットカード番号の登録が求められる場合もあります（その場合でも、利用開始後しばらくは無償であったり、一定期間内の利用が一定以内であれば無料、それ以上は利用した分に応じて課金されるような仕組みをとっている場合が多いです）。    
また、無償で利用可能なサイトでは、悪用されないように、個人情報に加え、何のために利用するのかといった説明を記載させ、数日程度の審査を通じてアカウントが発行される場合もあります。   

今回の教材には、Twitter、IBM Watson、Yahooのプログラムを載せていますが、これらのプログラムを実行するためには、アカウントの登録が必要で、その際に各社に皆さんの個人情報を登録する必要があります。   
本授業の一環として皆さんの個人情報を企業に渡すことをお勧めすることは本意ではないので、   
**`WebDataProcessing?.ipynb ('?'は1から4の整数)`以外のファイルは今回は実行不要です。**   

一方、`pandas-datareader`のように、無償・アカウント不要でWeb APIサービスを提供してくれているサイトもあります。   
今回はその一つである[Livedoor天気情報のWether Hacks](http://weather.livedoor.com/weather_hacks/)を利用させていただくことにしましょう。   
くれぐれもwhileループなどで高頻度でアクセスしないでくださいね！


## 1.  Libedoor天気情報の実行

今回は[Livedoor天気情報のWether Hacksの[お天気Webサービス（REST）](http://weather.livedoor.com/weather_hacks/webservice)」を使います。   
以下でこのサイトに記載されている情報を参照します（2019/7アクセス）。   

```
お天気Webサービス（Livedoor Weather Web Service / LWWS）は、現在全国142カ所の今日・明日・あさっての天気予報・予想気温と都道府県の天気概況情報を提供しています。

```
詳しくは[お天気Webサービス仕様](http://weather.livedoor.com/weather_hacks/webservice)をご覧ください。   
値域(city)を指定するパラメータは、[こちら](http://weather.livedoor.com/forecast/rss/primary_area.xml)から見つけることができます。   
```
<pref title="東京都">
<warn title="警報・注意報" source="http://weather.livedoor.com/forecast/rss/warn/13.xml"/>
<city title="東京" id="130010" source="http://weather.livedoor.com/forecast/rss/area/130010.xml"/>
<city title="大島" id="130020" source="http://weather.livedoor.com/forecast/rss/area/130020.xml"/>
<city title="八丈島" id="130030" source="http://weather.livedoor.com/forecast/rss/area/130030.xml"/>
<city title="父島" id="130040" source="http://weather.livedoor.com/forecast/rss/area/130040.xml"/>
</pref>
```
とあるので、東京のIDは「130010」であることがわかります。

「東京都・東京の天気」を取得する場合   
下記URLにアクセスしてJSONデータを取得します。   
```
基本URL + 東京のID（130010）
http://weather.livedoor.com/forecast/webservice/json/v1?city=130010
```

結果はjson形式で返されます。


In [ ]:
import requests

# 東京都の
url = 'http://weather.livedoor.com/forecast/webservice/json/v1?city=130010'
api_data = requests.get(url).json()


jsonモジュールを使って、結果を出力してみましょう。  
まずはデータの全貌です。

In [ ]:
import json

print(json.dumps(api_data, indent=2, ensure_ascii=False))

要素を取り出してみましょう。

In [ ]:
import json

for d in api_data['forecasts']:
    if '今日' in d['dateLabel']:
        print('今日の天気：', d['telop'])
    elif '明日' in d['dateLabel']:
        print('明日の天気：', d['telop'])
    elif '明後日' in d['dateLabel']:
        print('明後日の天気：', d['telop'])

print('--------- 天気の概要 -------------')
print(api_data['description']["text"].replace('\n\n','\n'))


## 2. 国立国会図書館サーチ API

[国立国会図書館](https://iss.ndl.go.jp/information/api/)は、「検索用API」「ハーベスト用API」「書影API」の3種類のAPIを提供しています。   
同ページによると、2020年7月現在、「(2) 個人・非営利団体等の方＞データ利用により利益を得ない場合」は利用申請は不要とあります。   
規約は変わる可能性があるのでご注意ください。

### 検索用API

検索条件（検索クエリとも呼びます）はqueryで指定しています。   
[国立国会図書館 API仕様の概要](https://iss.ndl.go.jp/information/api/)を参照し、いろいろと変えてみましょう。


In [ ]:
import requests
import urllib.parse

# クエリは題名（title)に「東京大学」を含み、2018年以降の本とします
query = urllib.parse.quote('title="東京大学" AND from="2018"')

# urlにqueryをつないでリクエストを送ります
# 「maximumRecords=10」は、検索結果の書誌情報を10件返してほしいという意味です。
url = 'https://iss.ndl.go.jp/api/sru?operation=searchRetrieve&maximumRecords=10&query='+query
res = requests.get(url)


In [ ]:
# 送り返されてきたデータをテキスト形式で表示してみましょう。
# これはXML形式です。
print(res.text)

XML形式をそのままでは見づらいので、XMLパーサを使ってパースします。   
ここでは`lxml`というパーサを使います。   
検索結果として、返ってきた10件の本について、その題名（title)、著者（creator, 情報がない可能性がある）、言語（情報がない可能性がある）を出力してみましょう。

In [ ]:
from lxml import etree
 
root = etree.fromstring(res.content)

num = 1
for record in root.findall('.//recordData', root.nsmap):
    print(record.find('recordPosition'))
    print('==== Search Result: '+str(num)+' ================')
    num += 1
    book = etree.fromstring(record.text)
    print("title:" + str(book.find('./{*}title').text)) # 題名
    if book.find(('./{*}creator')) is not None: # 著者名（情報がない可能性がある）
        print("creator:" + str(book.find('./{*}creator').text))
    if book.find(('./{*}language')) is not None: # 著者名（情報がない可能性がある）
        print("language:" + str(book.find('./{*}language').text))
